<!--
    Licensed to the Apache Software Foundation (ASF) under one
    or more contributor license agreements.  See the NOTICE file
    distributed with this work for additional information
    regarding copyright ownership.  The ASF licenses this file
    to you under the Apache License, Version 2.0 (the
    "License"); you may not use this file except in compliance
    with the License.  You may obtain a copy of the License at

      http://www.apache.org/licenses/LICENSE-2.0

    Unless required by applicable law or agreed to in writing,
    software distributed under the License is distributed on an
    "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY
    KIND, either express or implied.  See the License for the
    specific language governing permissions and limitations
    under the License.
-->

# Interactive Beam Examples

In [1]:
import apache_beam as beam
from apache_beam.runners.interactive import interactive_runner
p = beam.Pipeline(interactive_runner.InteractiveRunner())

/usr/local/google/home/ningk/workspace/beam/sdks/python/apache_beam/__init__.py:84: UserWarning: Some syntactic constructs of Python 3 are not yet fully supported by Apache Beam.
  'Some syntactic constructs of Python 3 are not yet fully supported by '


In [2]:
init_pcoll = p |  'Init Create' >> beam.Create(range(10))

import logging
logging.warning(id(p))
logging.warning('init_pcoll %s', id(init_pcoll))


squares = init_pcoll | 'Square' >> beam.Map(lambda x: x*x)
logging.warning(id(p))
logging.warning('squares %s', id(squares))

cubes = init_pcoll | 'Cube' >> beam.Map(lambda x: x**3)
logging.warning(id(p))
logging.warning('cubes %s', id(cubes))

In [3]:
result = p.run()
result.wait_until_finish()

In [ ]:
init_list = list(range(10))
squares_list = list(result.get(squares))
cubes_list = list(result.get(cubes))

squares_list.sort()
cubes_list.sort()

!pip install matplotlib

%matplotlib inline
from matplotlib import pyplot as plt
plt.scatter(init_list, squares_list, label='squares', color='red')
plt.scatter(init_list, cubes_list, label='cubes', color='blue')
plt.legend(loc='upper left')
plt.show()

In [ ]:
class AverageFn(beam.CombineFn):
  def create_accumulator(self):
    return (0.0, 0)

  def add_input(self, sum_count, input):
    (sum, count) = sum_count
    return sum + input, count + 1

  def merge_accumulators(self, accumulators):
    sums, counts = zip(*accumulators)
    return sum(sums), sum(counts)

  def extract_output(self, sum_count):
    (sum, count) = sum_count
    return sum / count if count else float('NaN')

In [ ]:
average_square = squares | 'Average Square' >> beam.CombineGlobally(AverageFn())
average_cube = cubes | 'Average Cube' >> beam.CombineGlobally(AverageFn())


In [ ]:
result = p.run()

In [4]:
from apache_beam.runners.dataflow import dataflow_runner
from apache_beam.options.pipeline_options import PipelineOptions
from apache_beam.options.pipeline_options import GoogleCloudOptions
from apache_beam.options.pipeline_options import StandardOptions
options = PipelineOptions()
gcloud_options = options.view_as(GoogleCloudOptions)
gcloud_options.project = 'ningk-test-project'
gcloud_options.job_name = 'test-job'
gcloud_options.staging_location = 'gs://ningk-test'
gcloud_options.temp_location = 'gs://ningk-test'
gcloud_options.sdk_location = 
options.view_as(StandardOptions).runner = 'DataflowRunner'
dataflow_runner.DataflowRunner().run_pipeline(p, options)

RuntimeError: Full traceback: Traceback (most recent call last):
  File "/usr/local/google/home/ningk/workspace/beam/sdks/python/apache_beam/utils/processes.py", line 83, in check_output
    out = subprocess.check_output(*args, **kwargs)
  File "/usr/lib/python3.6/subprocess.py", line 336, in check_output
    **kwargs).stdout
  File "/usr/lib/python3.6/subprocess.py", line 418, in run
    output=stdout, stderr=stderr)
subprocess.CalledProcessError: Command '['/usr/local/google/home/ningk/IdeaProjects/beam_p3_venv/bin/python', '-m', 'pip', 'download', '--dest', '/tmp/tmp3k7gf_8y', 'apache-beam==2.16.0.dev0', '--no-deps', '--no-binary', ':all:']' returned non-zero exit status 1.
 
 Pip install failed for package: apache-beam==2.16.0.dev0         
 Output from execution of subprocess: b'Collecting apache-beam==2.16.0.dev0\n'